Customer Analysis

In [38]:
#import the necessary libraries
import pandas as pd
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

In [39]:
#read the data uisng pandas 
df = pd.read_excel('SStore_data.xls')

In [40]:
#visualize the data for clarity
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [41]:
# define NOW - Whic is a date. it can also be todays date 
NOW = dt.datetime(2017,12,31)

In [42]:
# Make the date_placed column datetime
df['Order Date']= pd.to_datetime(df['Order Date'])


In [43]:
#group the data by customer name, creat new columns Order date, the 
rfmTable = df.groupby('Customer Name').agg({'Order Date': lambda x: (NOW - x.max()).days, # Recency
                                        'Order ID': lambda x: len(x),      # Frequency
                                        'Sales': lambda x: x.sum()})        # Monetary Value



In [44]:
#Change col names of our new table 
rfmTable['Order Date'] = rfmTable['Order Date'].astype(int)
rfmTable.rename(columns={'Order Date': 'Recency', 
                         'Order ID': 'Frequency', 
                         'Sales': 'MoneyValue'}, inplace=True)

In [45]:
#Quick visual

rfmTable.head()

,Recency,Frequency,MoneyValue
Customer Name,,,
Aaron Bergman,416,6,886.156
Aaron Hawkins,13,11,1744.700
Aaron Smayling,89,10,3050.692
Adam Bellavance,55,18,7755.620
Adam Hart,35,20,3250.337


In [46]:
#Quick look at customer name, view history of purchase
Cust1 = df[df['Customer Name']=='Claire Gute']
Cust1

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940,3,0.0,219.5820
5491,5492,CA-2017-164098,2017-01-26,2017-01-27,First Class,CG-12520,Claire Gute,Consumer,United States,Houston,...,77070,Central,OFF-ST-10000615,Office Supplies,Storage,"SimpliFile Personal File, Black Granite, 15w x...",18.160,2,0.2,1.8160
6877,6878,US-2015-123918,2015-10-15,2015-10-15,Same Day,CG-12520,Claire Gute,Consumer,United States,Dallas,...,75217,Central,FUR-FU-10004952,Furniture,Furnishings,C-Line Cubicle Keepers Polyproplyene Holder w/...,131.376,6,0.6,-95.2476
6878,6879,US-2015-123918,2015-10-15,2015-10-15,Same Day,CG-12520,Claire Gute,Consumer,United States,Dallas,...,75217,Central,OFF-PA-10003001,Office Supplies,Paper,Xerox 1986,5.344,1,0.2,1.8704


## Determining RFM Quartiles

In [47]:
#set the quatile values 
quant = rfmTable.quantile(q=[0.25,0.5,0.75])

In [48]:
#visualize the values
quant 

,Recency,Frequency,MoneyValue
0.25,31.0,8.0,1146.050
0.50,76.0,12.0,2256.394
0.75,184.0,16.0,3785.276


In [49]:
#copy to dictionary
quant = quant.to_dict()


In [51]:
#visualize once more
quant

{'Recency': {0.25: 31.0, 0.5: 76.0, 0.75: 184.0},
 'Frequency': {0.25: 8.0, 0.5: 12.0, 0.75: 16.0},
 'MoneyValue': {0.25: 1146.0500000000002, 0.5: 2256.394, 0.75: 3785.276}}

## Creating the RFM segmentation table

In [52]:
rfmSeg = rfmTable

In [54]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [55]:
rfmSeg['R_Quartile'] = rfmSeg['Recency'].apply(RClass, args=('Recency',quant,))
rfmSeg['F_Quartile'] = rfmSeg['Frequency'].apply(FMClass, args=('Frequency',quant,))
rfmSeg['M_Quartile'] = rfmSeg['MoneyValue'].apply(FMClass, args=('MoneyValue',quant,))

In [56]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)

In [57]:
rfmSeg.sort_values(by=['RFMClass'])

,Recency,Frequency,MoneyValue,R_Quartile,F_Quartile,M_Quartile,RFMClass
Customer Name,,,,,,,
Kunst Miller,29,28,4909.472,1,1,1,111
Natalie DeCherney,28,18,4326.139,1,1,1,111
James Galang,2,18,6366.392,1,1,1,111
William Brown,21,37,6160.102,1,1,1,111
Kelly Lampkin,17,22,5016.488,1,1,1,111
...,...,...,...,...,...,...,...
Susan Gilcrest,227,5,47.946,4,4,4,444
Julia Dunbar,301,6,695.442,4,4,4,444
Karen Seio,296,4,88.472,4,4,4,444


### Understanding


From the RFMClass column it can be clear that the three digit numbers are the representing three values of the RFM scale. 
from businesss perspective.

The high Recency value is bad, and the low recency is better off. 1>2>3>3
The high Frequency is good this means the customer is a return customer. 
While the High monetary customer is of special value. this are the value buyers. The produce high yeald.